# Colab-traiNNer-inference

victorca25's BasicSR fork: [victorca25/traiNNer](https://github.com/victorca25/traiNNer)

My fork: [styler00dollar/Colab-traiNNer](https://github.com/styler00dollar/Colab-traiNNer)

Currently only a minimalistic example for inpainting. 

In [ ]:
!nvidia-smi

In [ ]:
#@title Install dependencies (only the most basic ones, install others if needed)
%cd /content/
# create empty folders
!mkdir /content/input
!mkdir /content/output

!git clone https://github.com/styler00dollar/Colab-traiNNer
!pip install omegaconf piq wget adamp efficientnet_pytorch tensorboardX vit-pytorch swin-transformer-pytorch madgrad timm pillow-avif-plugin kornia
!pip uninstall pytorch_lightning -y
!pip install git+https://github.com/styler00dollar/pytorch-lightning.git@remotes/origin/release/1.4.x

!pip install tfrecord tensorboardX -U
!pip install crc32c==2.0 # downgrade to surpress tensorboardX warnings

In [ ]:
#@title inpaint inference (cuda)
#@markdown Put files into input folder `input.png / input_mask.png`
#@markdown and manually specify model and its parameters
import glob
import cv2
import os
from tqdm import tqdm
import torch
from torchvision.utils import save_image

rootdir = "/content/input" #@param
outdir = "/content/output/" #@param
fp16 = True #@param {type:"boolean"}
model_path = "/content/drive/MyDrive/model.pth" #@param
files = glob.glob(rootdir + '/**/*.png', recursive=True)

# select and load a model
%cd /content/Colab-traiNNer/code/arch
from lama_arch import FFCResNetGenerator
netG = FFCResNetGenerator(4, 3)
netG.load_state_dict(torch.load(model_path))
netG.cuda().eval()
if fp16:
  netG.half()
%cd /content/

# create list with only image files
inference_files = []
for i in files:
  if "_mask" not in i:
    inference_files.append(i)

for f in tqdm(inference_files):
  image = cv2.imread(f)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  mask = cv2.imread(os.path.join(rootdir, os.path.splitext(os.path.basename(f))[0] + "_mask.png"), cv2.IMREAD_GRAYSCALE)
  # forcing loaded mask to either be 0 or 255
  _,mask = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
  image[mask==255] = [0,0,0]
  image = torch.from_numpy(image).cuda().unsqueeze(0).permute(0,3,1,2)/255
  mask = torch.from_numpy(mask).cuda().unsqueeze(0).unsqueeze(0)/255
  if fp16:
    image = image.half()
    mask = mask.half()
  
  mask = 1-mask # inverting mask
  out = netG(image, mask)
  out = out*(1-mask)+image*mask
  save_image(out, os.path.join(outdir, os.path.basename(f)))

  # OOM hotfix
  torch.cuda.empty_cache()

In [ ]:
#@title delete input/output
%cd /content/
!sudo rm -rf /content/input
!mkdir /content/input
!sudo rm -rf /content/output
!mkdir /content/output